# Part C 

In [1]:
import sys
import random
from pprint import pprint as pp
random.seed(42)
sys.version

'3.7.4 (default, Oct 15 2019, 22:29:14) \n[GCC 7.4.0]'

In [2]:
import neo4j
import py2neo
print(neo4j.__version__)
print(py2neo.__version__)

1.7.6
4.3.0


In [3]:
from neo4j import GraphDatabase
from py2neo import Graph

# instantiate drivers
NEO4J_URI="bolt://localhost:7687"
gdb = GraphDatabase.driver(uri=NEO4J_URI, auth=None)
graph = Graph(NEO4J_URI)

The graph has the following structure

![graph](./schemas/dblp_slim_after/graph.png)

In [16]:
graph.run("CALL algo.list();").data()[:1]

[{'name': 'algo.allShortestPaths.stream',
  'description': "CALL algo.allShortestPaths.stream(weightProperty:String{nodeQuery:'labelName', relationshipQuery:'relationshipName', defaultValue:1.0, concurrency:4}) YIELD sourceNodeId, targetNodeId, distance - yields a stream of {sourceNodeId, targetNodeId, distance}",
  'signature': 'algo.allShortestPaths.stream(propertyName :: STRING?, config = {} :: MAP?) :: (sourceNodeId :: INTEGER?, targetNodeId :: INTEGER?, distance :: FLOAT?)',
  'type': 'procedure'}]

## PageRank

In [11]:
q_page_rank ="""
CALL algo.pageRank.stream(
    'MATCH (a:Article) 
     WHERE exists ((a)−[:CITED_BY]−>()) 
     RETURN ID(a) AS id',
    'MATCH (a1:Article)−[:CITED_BY]−>(a2:Article)
     RETURN id (a1) AS source, ID(a2) AS target',
    {graph : 'cypher', iterations:20, dampingFactor:0.85}) 
YIELD nodeId, score 
MATCH (a:Article) WHERE ID(a) = nodeId
RETURN a.title AS title, score 
ORDER BY score DESC;"""

graph.run(q_page_rank).data()[:5]

[{'title': 'Template-directed molecular assembly on silicon carbide nanomesh: comparison between CuPc and pentacene.',
  'score': 1.3380366005469115},
 {'title': 'Influence of investment, disinfection, and storage on the microhardness of ocular resins.',
  'score': 1.2711560409981755},
 {'title': 'Fast Runtime Block Cyclic Data Redistribution on Multiprocessors',
  'score': 1.2702508100308478},
 {'title': 'Replication of the range of native anterior cruciate ligament fiber length change behavior achieved by different grafts: measurement using computer-assisted navigation.',
  'score': 1.2594134361017493},
 {'title': 'Evidence for maize (Zea mays) in the Late Archaic (3000-1800 B.C.) in the Norte Chico region of Peru.',
  'score': 1.2585701291449365}]

In [ ]:
## 